In [1]:
import sys
sys.path.insert(0, '/home/anish/ASC_ML')

In [2]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [3]:
from ASC_ML import dataframe_extractor as de
from ASC_ML import model_generation as model_gen
from ASC_ML import model_gen_train_test as testing
from dask_ml.preprocessing import LabelEncoder
# from dask_ml.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

2022-06-15 17:59:32.210846: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [4]:
from ASC_ML import multiple_model_gen_v1 as multiple

In [5]:
# import dask.dataframe as dd
# import pandas as pd
# data = dd.from_pandas(pd.Series(['a', 'a', 'b'], dtype='category'),
#                      npartitions=1)
# le = LabelEncoder()
# le.fit_transform(data).compute()
# print(type(le.fit_transform(data).compute()))

In [6]:
#For House Prediction Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=train.csv
directory1 = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv"


# returns list of dask dataframe [singular_df_x, singular_df_y] or [train_df_x, train_df_y, test_df_x, test_df_y]
dataset_list = de.DataframeExtractor_csv(directory1, label_names = ["SalePrice"]).df_list

print("Train Dataset X Columns")
dataset_list[0].head()

Reading single csv from /home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv
Train Dataset X Columns


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,12.0,2008.0,WD,Normal


In [7]:
dataset_list[1].head()

,SalePrice
0,208500.0
1,181500.0
2,223500.0
3,140000.0
4,250000.0


In [8]:
train_x = dataset_list[0].copy()

In [9]:
train_x = train_x.fillna(0)

In [10]:
train_x.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [11]:
train_x['MSSubClass'] = train_x['MSSubClass'].apply(str)

train_x['OverallCond'] = train_x['OverallCond'].astype(str)

train_x['YrSold'] = train_x['YrSold'].astype(str)
train_x['MoSold'] = train_x['MoSold'].astype(str)
train_x.head()

/home/anish/anaconda3/envs/tfgpu/lib/python3.9/site-packages/dask/dataframe/core.py:3946: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('MSSubClass', 'object'))

  warnings.warn(meta_warning(meta))


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [12]:
# a = train_x['LotShape']

In [13]:
# le = LabelEncoder() 
# le.fit_transform(a).compute()
# train_x['LotShape'] = le.fit_transform(train_x['LotShape'])

In [14]:
# train_x.head()

In [15]:
def keyval(allkeys):
    i = 0
    keyvalpairs = list()
    for key in allkeys:
        keyvalpairs.append((key,i))
        i+=1
    return dict(keyvalpairs)

In [16]:
# cols = ['FireplaceQu', 'LotShape']
cols = ['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold']
keys = {}
for col in cols:
    key = train_x[col].unique()
    keyvalpairs = keyval(key)
    keys.update({col:keyvalpairs})


In [17]:
keys

{'FireplaceQu': {0: 0, 'TA': 1, 'Gd': 2, 'Fa': 3, 'Ex': 4, 'Po': 5},
 'BsmtQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 0: 3, 'Fa': 4},
 'BsmtCond': {'TA': 0, 'Gd': 1, 0: 2, 'Fa': 3, 'Po': 4},
 'GarageQual': {'TA': 0, 'Fa': 1, 'Gd': 2, 0: 3, 'Ex': 4, 'Po': 5},
 'GarageCond': {'TA': 0, 'Fa': 1, 0: 2, 'Gd': 3, 'Po': 4, 'Ex': 5},
 'ExterQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'ExterCond': {'TA': 0, 'Gd': 1, 'Fa': 2, 'Po': 3, 'Ex': 4},
 'HeatingQC': {'Ex': 0, 'Gd': 1, 'TA': 2, 'Fa': 3, 'Po': 4},
 'PoolQC': {0: 0, 'Ex': 1, 'Fa': 2, 'Gd': 3},
 'KitchenQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'BsmtFinType1': {'GLQ': 0,
  'ALQ': 1,
  'Unf': 2,
  'Rec': 3,
  'BLQ': 4,
  0: 5,
  'LwQ': 6},
 'BsmtFinType2': {'Unf': 0,
  'BLQ': 1,
  0: 2,
  'ALQ': 3,
  'Rec': 4,
  'LwQ': 5,
  'GLQ': 6},
 'Functional': {'Typ': 0,
  'Min1': 1,
  'Maj1': 2,
  'Min2': 3,
  'Mod': 4,
  'Maj2': 5,
  'Sev': 6},
 'Fence': {0: 0, 'MnPrv': 1, 'GdWo': 2, 'GdPrv': 3, 'MnWw': 4},
 'BsmtExposure': {'No': 0, 'Gd': 1, 'Mn': 2, '

In [18]:
train_X = train_x.replace(keys).copy()

In [19]:
train_X['FireplaceQu'].unique().compute()

0    0
1    1
2    2
3    3
4    4
5    5
Name: FireplaceQu, dtype: int64

In [20]:
train_X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,0,RL,65.0,8450.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,0,WD,Normal
1,2.0,1,RL,80.0,9600.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,1,1,WD,Normal
2,3.0,0,RL,68.0,11250.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2,0,WD,Normal
3,4.0,2,RL,60.0,9550.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,2,WD,Abnorml
4,5.0,0,RL,84.0,14260.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,3,0,WD,Normal


In [21]:
cols_to_remove = []
for col in train_X.columns:
#     print(type(col))
    try:
        a = train_X[col].astype(float).compute()
    except ValueError:
#         print('Couldn\'t covert %s to float' % col)
        cols_to_remove.append(col)
        pass

# keep only the columns in df that do not contain string
train_X = train_X[[col for col in train_X.columns if col not in cols_to_remove]]

In [22]:
print(cols_to_remove)

['MSZoning', 'LandContour', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition']


In [23]:
train_X = train_X.drop(["Id"], axis = 1)

In [24]:
# tsklearn.preprocessing.MinMaxScalerrain_X.head()

In [25]:
len(train_X.columns)

58

In [26]:
len(dataset_list[1].columns)

1

In [27]:
scaler_x = MinMaxScaler()
scaler_x.fit(train_X)
train_X_scaled = scaler_x.transform(train_X)

scaler_y = MinMaxScaler()
scaler_y.fit(dataset_list[1])
train_Y_scaled = scaler_y.transform(dataset_list[1])

In [28]:
# col = train_X.columns.to_list()
# scaler_d = MinMaxScaler()
# train_X.columns = list(train_X.columns)
# scaled_train_X = scaler_d.fit_transform(train_X[col])

In [29]:
train_X_scaled
# train_Y_scaled

array([[0.        , 0.20766773, 0.0334198 , ..., 0.        , 0.        ,
        0.        ],
       [0.07142857, 0.25559105, 0.03879502, ..., 0.        , 0.09090909,
        0.25      ],
       [0.        , 0.2172524 , 0.04650728, ..., 0.        , 0.18181818,
        0.        ],
       ...,
       [0.14285714, 0.21086262, 0.03618687, ..., 0.16129032, 0.09090909,
        1.        ],
       [0.07142857, 0.2172524 , 0.03934189, ..., 0.        , 0.63636364,
        1.        ],
       [0.07142857, 0.23961661, 0.04037019, ..., 0.        , 1.        ,
        0.        ]])

In [30]:
TRAIN = testing.Model_Parallel_Train_Test(train_X_scaled, train_Y_scaled, 58, 1, None)

In [31]:
TRAIN

In [32]:
model = TRAIN.get_models()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer_dense_4_4_4 (InputL [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_8_8_8 (InputL [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_16_16_16 (Inp [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_32_32_32 (Inp [(None, 58)]         0                                            
______________________________________________________________________________________________

2022-06-15 17:59:42.486473: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-15 17:59:42.486699: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-15 17:59:42.487084: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-15 17:59:42.487820: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.695GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2022-06-15 17:59:42.487926: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-15 17:59:42.492615: I tensorflow/stream_executor/platform/de

In [33]:
TRAIN.train_models()

Epoch 1/100


2022-06-15 17:59:43.214602: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-15 17:59:43.231674: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2899885000 Hz


 1/12 [=>............................] - ETA: 14s - loss: 0.4101 - output_layer_dense_4_4_4_loss: 0.0532 - output_layer_dense_8_8_8_loss: 0.0216 - output_layer_dense_16_16_16_loss: 0.1793 - output_layer_dense_32_32_32_loss: 0.0509 - output_layer_dense_64_64_64_loss: 0.0424 - output_layer_dense_128_128_128_loss: 0.0627 - output_layer_dense_4_4_4_mean_absolute_error: 0.1984 - output_layer_dense_8_8_8_mean_absolute_error: 0.1098 - output_layer_dense_16_16_16_mean_absolute_error: 0.3899 - output_layer_dense_32_32_32_mean_absolute_error: 0.1947 - output_layer_dense_64_64_64_mean_absolute_error: 0.1741 - output_layer_dense_128_128_128_mean_absolute_error: 0.2140

2022-06-15 17:59:44.431961: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


12/12 [==============================] - 1s 4ms/step - loss: 0.2792 - output_layer_dense_4_4_4_loss: 0.0473 - output_layer_dense_8_8_8_loss: 0.0166 - output_layer_dense_16_16_16_loss: 0.1401 - output_layer_dense_32_32_32_loss: 0.0276 - output_layer_dense_64_64_64_loss: 0.0189 - output_layer_dense_128_128_128_loss: 0.0287 - output_layer_dense_4_4_4_mean_absolute_error: 0.1854 - output_layer_dense_8_8_8_mean_absolute_error: 0.0962 - output_layer_dense_16_16_16_mean_absolute_error: 0.3356 - output_layer_dense_32_32_32_mean_absolute_error: 0.1301 - output_layer_dense_64_64_64_mean_absolute_error: 0.1050 - output_layer_dense_128_128_128_mean_absolute_error: 0.1304
Epoch 2/100
12/12 [==============================] - 0s 4ms/step - loss: 0.1045 - output_layer_dense_4_4_4_loss: 0.0288 - output_layer_dense_8_8_8_loss: 0.0111 - output_layer_dense_16_16_16_loss: 0.0438 - output_layer_dense_32_32_32_loss: 0.0093 - output_layer_dense_64_64_64_loss: 0.0064 - output_layer_dense_128_128_128_loss: 0.00

...

12/12 [==============================] - 0s 4ms/step - loss: 0.0066 - output_layer_dense_4_4_4_loss: 0.0019 - output_layer_dense_8_8_8_loss: 0.0015 - output_layer_dense_16_16_16_loss: 0.0016 - output_layer_dense_32_32_32_loss: 0.0010 - output_layer_dense_64_64_64_loss: 4.0118e-04 - output_layer_dense_128_128_128_loss: 1.6677e-04 - output_layer_dense_4_4_4_mean_absolute_error: 0.0297 - output_layer_dense_8_8_8_mean_absolute_error: 0.0253 - output_layer_dense_16_16_16_mean_absolute_error: 0.0277 - output_layer_dense_32_32_32_mean_absolute_error: 0.0223 - output_layer_dense_64_64_64_mean_absolute_error: 0.0140 - output_layer_dense_128_128_128_mean_absolute_error: 0.0089
Epoch 47/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0069 - output_layer_dense_4_4_4_loss: 0.0020 - output_layer_dense_8_8_8_loss: 0.0016 - output_layer_dense_16_16_16_loss: 0.0018 - output_layer_dense_32_32_32_loss: 9.6876e-04 - output_layer_dense_64_64_64_loss: 3.7099e-04 - output_layer_dense_128_1

12/12 [==============================] - 0s 4ms/step - loss: 0.0060 - output_layer_dense_4_4_4_loss: 0.0019 - output_layer_dense_8_8_8_loss: 0.0013 - output_layer_dense_16_16_16_loss: 0.0016 - output_layer_dense_32_32_32_loss: 8.5320e-04 - output_layer_dense_64_64_64_loss: 2.5346e-04 - output_layer_dense_128_128_128_loss: 9.7404e-05 - output_layer_dense_4_4_4_mean_absolute_error: 0.0290 - output_layer_dense_8_8_8_mean_absolute_error: 0.0241 - output_layer_dense_16_16_16_mean_absolute_error: 0.0270 - output_layer_dense_32_32_32_mean_absolute_error: 0.0207 - output_layer_dense_64_64_64_mean_absolute_error: 0.0115 - output_layer_dense_128_128_128_mean_absolute_error: 0.0070
Epoch 58/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0066 - output_layer_dense_4_4_4_loss: 0.0019 - output_layer_dense_8_8_8_loss: 0.0016 - output_layer_dense_16_16_16_loss: 0.0018 - output_layer_dense_32_32_32_loss: 8.9348e-04 - output_layer_dense_64_64_64_loss: 2.7133e-04 - output_layer_dense_1

12/12 [==============================] - 0s 4ms/step - loss: 0.0051 - output_layer_dense_4_4_4_loss: 0.0016 - output_layer_dense_8_8_8_loss: 0.0012 - output_layer_dense_16_16_16_loss: 0.0014 - output_layer_dense_32_32_32_loss: 6.9008e-04 - output_layer_dense_64_64_64_loss: 2.0175e-04 - output_layer_dense_128_128_128_loss: 6.2654e-05 - output_layer_dense_4_4_4_mean_absolute_error: 0.0273 - output_layer_dense_8_8_8_mean_absolute_error: 0.0229 - output_layer_dense_16_16_16_mean_absolute_error: 0.0244 - output_layer_dense_32_32_32_mean_absolute_error: 0.0188 - output_layer_dense_64_64_64_mean_absolute_error: 0.0104 - output_layer_dense_128_128_128_mean_absolute_error: 0.0056
Epoch 69/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0059 - output_layer_dense_4_4_4_loss: 0.0018 - output_layer_dense_8_8_8_loss: 0.0015 - output_layer_dense_16_16_16_loss: 0.0016 - output_layer_dense_32_32_32_loss: 7.5113e-04 - output_layer_dense_64_64_64_loss: 2.0448e-04 - output_layer_dense_1

12/12 [==============================] - 0s 4ms/step - loss: 0.0054 - output_layer_dense_4_4_4_loss: 0.0018 - output_layer_dense_8_8_8_loss: 0.0013 - output_layer_dense_16_16_16_loss: 0.0014 - output_layer_dense_32_32_32_loss: 6.1743e-04 - output_layer_dense_64_64_64_loss: 1.5690e-04 - output_layer_dense_128_128_128_loss: 4.2782e-05 - output_layer_dense_4_4_4_mean_absolute_error: 0.0274 - output_layer_dense_8_8_8_mean_absolute_error: 0.0229 - output_layer_dense_16_16_16_mean_absolute_error: 0.0237 - output_layer_dense_32_32_32_mean_absolute_error: 0.0173 - output_layer_dense_64_64_64_mean_absolute_error: 0.0087 - output_layer_dense_128_128_128_mean_absolute_error: 0.0046    
Epoch 80/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0053 - output_layer_dense_4_4_4_loss: 0.0017 - output_layer_dense_8_8_8_loss: 0.0013 - output_layer_dense_16_16_16_loss: 0.0014 - output_layer_dense_32_32_32_loss: 6.4577e-04 - output_layer_dense_64_64_64_loss: 1.5125e-04 - output_layer_den

12/12 [==============================] - 0s 4ms/step - loss: 0.0059 - output_layer_dense_4_4_4_loss: 0.0019 - output_layer_dense_8_8_8_loss: 0.0015 - output_layer_dense_16_16_16_loss: 0.0016 - output_layer_dense_32_32_32_loss: 5.6661e-04 - output_layer_dense_64_64_64_loss: 1.6007e-04 - output_layer_dense_128_128_128_loss: 4.7473e-05 - output_layer_dense_4_4_4_mean_absolute_error: 0.0272 - output_layer_dense_8_8_8_mean_absolute_error: 0.0227 - output_layer_dense_16_16_16_mean_absolute_error: 0.0234 - output_layer_dense_32_32_32_mean_absolute_error: 0.0168 - output_layer_dense_64_64_64_mean_absolute_error: 0.0090 - output_layer_dense_128_128_128_mean_absolute_error: 0.0051
Epoch 91/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0047 - output_layer_dense_4_4_4_loss: 0.0016 - output_layer_dense_8_8_8_loss: 0.0011 - output_layer_dense_16_16_16_loss: 0.0013 - output_layer_dense_32_32_32_loss: 5.4188e-04 - output_layer_dense_64_64_64_loss: 1.5339e-04 - output_layer_dense_1

In [34]:
m = multiple.Multiple_Model_Gen(train_X_scaled, train_Y_scaled, 50, 128, input_shape = 58, min_no_layers = 3, max_no_layers = 3, model_per_batch = 5)

In [35]:
m.get_all_models()

Model: "dense_16_16_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer_dense_16_16_16 ( [(None, 58)]              0         
_________________________________________________________________
layer1_dense_16_16_16 (Dense (None, 16)                944       
_________________________________________________________________
layer2_dense_16_16_16 (Dense (None, 16)                272       
_________________________________________________________________
layer3_dense_16_16_16 (Dense (None, 16)                272       
_________________________________________________________________
output_layer_dense_16_16_16  (None, 1)                 17        
Total params: 1,505
Trainable params: 1,505
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50


ValueError: in user code:

    /home/anish/anaconda3/envs/tfgpu/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/anish/anaconda3/envs/tfgpu/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/anish/anaconda3/envs/tfgpu/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/anish/anaconda3/envs/tfgpu/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/anish/anaconda3/envs/tfgpu/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/anish/an